In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import defaultdict
import string
import tensorflow as tf
import re
import os
import time
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
ENCODER_LEN = 100
DECODER_LEN = 100
BATCH_SIZE = 64
BUFFER_SIZE = BATCH_SIZE*4

In [3]:
import pandas as pd
 
# Passing the TSV file to
# read_csv() function
# with tab separator
# This function will
# read data from file
interviews_df = pd.read_csv('C:/Users/gouth/Downloads/human_annotations.tsv', sep='\t')


train_file_path = 'C:/Users/gouth/Downloads/human_annotations.tsv'

 
with open(train_file_path, 'r', encoding='utf-8') as file:
    data = file.readlines()
    
data_split = [line.strip().split('\t') for line in data]

df = pd.DataFrame(data_split, columns=['language_id', 'value','Source','Indic_Sentence','English_Sentence','Labse_Score','bucket'])


# printing data
df.head()

,language_id,value,Source,Indic_Sentence,English_Sentence,Labse_Score,bucket
0,language_id,value,Source,Indic Sentence,English Sentence,Labse Score,bucket
1,Gujarati,4,Wikipedia,અલગ અલગ વપરાશકર્તાઓ અને કમ્પ્યુટિંગ એન્વાયન્મે...,Office is produced in several versions targete...,0.7818931340999999,Marginally accept
2,Gujarati,4,Wikipedia,અલગ અલગ વપરાશકર્તાઓ અને કમ્પ્યુટિંગ એન્વાયન્મે...,Office is produced in several versions targete...,0.7818931340999999,Marginally accept
3,Gujarati,5,Wikipedia,અલગ અલગ વપરાશકર્તાઓ અને કમ્પ્યુટિંગ એન્વાયન્મે...,Office is produced in several versions targete...,0.7818931340999999,Marginally accept
4,Gujarati,3,NPTEL,ક્ષેત્ર ઘટાડો.,Area reduction.,0.8527,Definitely accept


In [4]:
df = df[1:]  # Removes the first row
df.head()

,language_id,value,Source,Indic_Sentence,English_Sentence,Labse_Score,bucket
1,Gujarati,4,Wikipedia,અલગ અલગ વપરાશકર્તાઓ અને કમ્પ્યુટિંગ એન્વાયન્મે...,Office is produced in several versions targete...,0.7818931340999999,Marginally accept
2,Gujarati,4,Wikipedia,અલગ અલગ વપરાશકર્તાઓ અને કમ્પ્યુટિંગ એન્વાયન્મે...,Office is produced in several versions targete...,0.7818931340999999,Marginally accept
3,Gujarati,5,Wikipedia,અલગ અલગ વપરાશકર્તાઓ અને કમ્પ્યુટિંગ એન્વાયન્મે...,Office is produced in several versions targete...,0.7818931340999999,Marginally accept
4,Gujarati,3,NPTEL,ક્ષેત્ર ઘટાડો.,Area reduction.,0.8527,Definitely accept
5,Gujarati,5,NPTEL,ક્ષેત્ર ઘટાડો.,Area reduction.,0.8527,Definitely accept


In [5]:
print("Number of rows:", df.shape[0])

Number of rows: 41453


In [6]:
train_df = df[df['language_id'] == 'Hindi']

# Displaying the extracted rows where only 'Hindi' is present in 'language_id'
train_df.head()

,language_id,value,Source,Indic_Sentence,English_Sentence,Labse_Score,bucket
4589,Hindi,2,Wikipedia,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4590,Hindi,1,Wikipedia,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4591,Hindi,3,Wikipedia,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4592,Hindi,3,Wikipedia,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4593,Hindi,3,NPTEL,"मान्य प्रश्न है हाँ, यह विरोधाभासी लग सकता है,...",And on the other hand I am saying that there i...,0.7154,Marginally reject


In [7]:
unique_buckets = df['bucket'].unique()
print("Unique values in 'bucket' column:")
print(unique_buckets)

Unique values in 'bucket' column:
['Marginally accept' 'Definitely accept' 'Marginally reject' None]


In [8]:
train_df.drop(['Source'],axis=1,inplace=True)
train_df.head()

C:\Users\gouth\AppData\Local\Temp\ipykernel_28792\2873094891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(['Source'],axis=1,inplace=True)


,language_id,value,Indic_Sentence,English_Sentence,Labse_Score,bucket
4589,Hindi,2,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4590,Hindi,1,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4591,Hindi,3,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4592,Hindi,3,"इनके अलावा ये रियासतें थी - बड़ोदा रियासत ,ग्व...",It was considered the most important of the Ma...,0.7216691375,Marginally reject
4593,Hindi,3,"मान्य प्रश्न है हाँ, यह विरोधाभासी लग सकता है,...",And on the other hand I am saying that there i...,0.7154,Marginally reject


In [9]:
eng = train_df['English_Sentence']
hind = train_df['Indic_Sentence']
eng = eng.apply(lambda x: "<SOS> " + str(x) + " <EOS>")
hind = hind.apply(lambda x: "<SOS> "+ x + " <EOS>")

In [10]:
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'
eng_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = filters, oov_token=oov_token)
hind_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = filters, oov_token=oov_token)
eng_tokenizer.fit_on_texts(eng)
hind_tokenizer.fit_on_texts(hind)
inputs = eng_tokenizer.texts_to_sequences(eng)
targets = hind_tokenizer.texts_to_sequences(hind)

In [11]:
ENCODER_VOCAB = len(eng_tokenizer.word_index) + 1
DECODER_VOCAB = len(hind_tokenizer.word_index) + 1
print(ENCODER_VOCAB, DECODER_VOCAB)

3861 5776


In [12]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=ENCODER_LEN, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=DECODER_LEN, padding='post', truncating='post')
inputs = tf.cast(inputs, dtype=tf.int64)
targets = tf.cast(targets, dtype=tf.int64)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [14]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [15]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights
    
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

In [16]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

In [17]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

In [34]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
            
        return x
    
class Decoder(tf.keras.layers.Layer):
        
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights
    

In [20]:
mask = (train_df['English_Sentence'].str.len()>20) & (train_df['English_Sentence'].str.len()<200)

In [21]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights

In [22]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1
EPOCHS = 10

In [41]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [42]:
learning_rate = CustomSchedule(d_model)

In [43]:
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [44]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [45]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy_function(tar_real, predictions))

In [46]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask

In [47]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=ENCODER_VOCAB,
    target_vocab_size=DECODER_VOCAB,
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate)

In [49]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [50]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [59]:
train_df.shape[0]

4616

In [68]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        if batch % 200 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
   
    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.0987 Accuracy 0.1982
Epoch 1 Loss 2.6124 Accuracy 0.2142
Time taken for 1 epoch: 152.40794277191162 secs

Epoch 2 Batch 0 Loss 2.6644 Accuracy 0.2144
Epoch 2 Loss 2.2993 Accuracy 0.2316
Time taken for 1 epoch: 154.853031873703 secs

Epoch 3 Batch 0 Loss 2.4823 Accuracy 0.2318
Epoch 3 Loss 2.0045 Accuracy 0.2499
Time taken for 1 epoch: 151.54523134231567 secs

Epoch 4 Batch 0 Loss 2.1762 Accuracy 0.2501
Epoch 4 Loss 1.7227 Accuracy 0.2689
Time taken for 1 epoch: 141.8621575832367 secs

Epoch 5 Batch 0 Loss 1.8524 Accuracy 0.2691
Saving checkpoint for epoch 5 at checkpoints\ckpt-5
Epoch 5 Loss 1.4444 Accuracy 0.2883
Time taken for 1 epoch: 138.06502652168274 secs

Epoch 6 Batch 0 Loss 1.6976 Accuracy 0.2886
Epoch 6 Loss 1.2070 Accuracy 0.3078
Time taken for 1 epoch: 155.81656789779663 secs

Epoch 7 Batch 0 Loss 1.4395 Accuracy 0.3080
Epoch 7 Loss 0.9966 Accuracy 0.3270
Time taken for 1 epoch: 148.44600296020508 secs

Epoch 8 Batch 0 Loss 1.1382 Accuracy 0.3272
Epoc

In [69]:
def evaluate(text):
    text = eng_tokenizer.texts_to_sequences([text])
    text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=ENCODER_LEN, 
                                                                   padding='post', truncating='post')

    encoder_input = tf.expand_dims(text[0], 0)

    decoder_input = [hind_tokenizer.word_index['<sos>']]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(DECODER_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == hind_tokenizer.word_index['<eos>']:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [70]:
def translate(eng_text):
    hind_text = evaluate(text=eng_text)[0].numpy()
    hind_text = np.expand_dims(hind_text[1:], 0)  
    return hind_tokenizer.sequences_to_texts(hind_text)[0]

In [73]:
translate("This is not a good idea")


'यह कोई अच्छी बात नहीं है'

In [80]:
translate("We are talking about Rajasthan")


'हम बात कर रहे हैं राजस्थान की।'